In [ ]:
import os
import pandas as pd
import numpy as np
from collections import defaultdict
import pywt
import matplotlib.pyplot as plt
from scipy.signal import iirnotch, freqz, filtfilt
from scipy.signal import butter, lfilter, lfilter_zi

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
eeg_dir = '/content/drive/My Drive/EEG_dataset/updated_data'


In [ ]:
def butter_bandpass(lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return b, a
def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    zi = lfilter_zi(b, a) * data[0]
    y, _ = lfilter(b, a, data, zi=zi)
    return y
def fft(data):
    number_of_points = len(filtered_eeg)
    fhat = np.fft.fft(filtered_eeg, number_of_points)
    PSD = fhat * np.conj(fhat) / number_of_points
    freq = (1/(0.004*number_of_points))*np.arange(number_of_points)
    L = np.arange(1,np.floor(number_of_points/2),dtype='int') 
    indices = PSD > 150
    PSDclean = PSD * indices
    fhat = indices * fhat
    return np.fft.ifft(fhat)   
def feature_extraction(data):
    mean = data.mean()
    std = np.std(data)
    ptp = np.ptp(data)
    var = np.var(data)
    print(round(mean)*10)
    return np.array([round(mean),round(std),round(ptp),round(var)])


In [ ]:
result_train = pd.DataFrame()
row=[]
labels_train=[]
result_test = pd.DataFrame()
labels_test=[]

In [ ]:
train_dir = os.path.join(eeg_dir, 'train')

for label_type in ['left', 'right', 'down', 'back', 'up','forward']:
	dir_name = os.path.join(train_dir, label_type)
	for fname in os.listdir(dir_name):
		with open(os.path.join(dir_name, fname)) as f:
			df = pd.read_csv(f)
			for column in df:
				band_pass_eeg = butter_bandpass_filter(data = df[column], lowcut = 1,highcut = 50, fs = 250, order = 3)
				b_notch, a_notch = iirnotch(w0=59.9250936329588, Q=20, fs=250)
				filtered_eeg = filtfilt(b_notch, a_notch, band_pass_eeg)
				fft_data = fft(filtered_eeg)
				feature =  feature_extraction(fft_data)
				row = np.concatenate((row,feature),axis=None)
		result_train= result_train.append(pd.DataFrame(row).T)
		labels_train.append(label_type)
		row=[]

#print(result_train)	

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:27: DeprecationWarning: The Python built-in `round` is deprecated for complex scalars, and will raise a `TypeError` in a future release. Use `np.round` or `scalar.round` instead.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:28: DeprecationWarning: The Python built-in `round` is deprecated for complex scalars, and will raise a `TypeError` in a future release. Use `np.round` or `scalar.round` instead.


(-0+0j)
0j
0j
0j
0j
0j
0j
(-0+0j)
0j
0j
0j
0j
(-0+0j)
(-0+0j)
0j
0j
0j
(-0+0j)
(-0+0j)
0j
(-90+0j)
0j
(-0+0j)
(-0+0j)
(-10+0j)
(-0+0j)
(-0+0j)
(-10+0j)
(-10+0j)
0j
0j
(-0+0j)
-0j
0j
0j
0j
0j
0j
-0j
(-0+0j)
0j
0j
0j
0j
(-10-0j)
(170+0j)
-0j
(-0+0j)
0j
(-120+0j)
-0j
(-10-0j)
(10+0j)
0j
(-0+0j)
(-0+0j)
0j
(-0+0j)
(-0+0j)
0j
0j
(-10+0j)
(-0+0j)
(-20-0j)
0j
(-0+0j)
0j
(-10+0j)
0j
(10+0j)
0j
(-10-0j)
-0j
0j
(-0+0j)
-0j
(-10+0j)
(-0+0j)
0j
-0j
(10+0j)
0j
0j
(-0+0j)
(-0+0j)
(-0+0j)
0j
(-0+0j)
(10+0j)
(-0+0j)
-0j
-0j
-0j
0j
-0j
(-0+0j)
-0j
(-0+0j)
0j
(-0+0j)
(-0+0j)
0j
0j
-0j
0j
0j
0j
0j
(-0+0j)
(-0+0j)
0j
(-0+0j)
0j
0j
-0j
(-0+0j)
0j
-0j
0j
-0j
(-0+0j)
-0j
0j
0j
(-0+0j)
(10+0j)
(-0+0j)
0j
(-0+0j)
(10+0j)
(-0+0j)
0j
0j
0j
0j
(-0+0j)
(-0+0j)
0j
(-0+0j)
(20+0j)
(10+0j)
-0j
(10+0j)
(-10+0j)
0j
(10+0j)
0j
0j
0j
(-0+0j)
0j
(-0+0j)
0j
(-0+0j)
(-0+0j)
-0j
(-0+0j)
(-0+0j)
0j
0j
(10+0j)
(-0+0j)
0j
(-0+0j)
0j
0j
(-0+0j)
(-0+0j)
(-0+0j)
(-0+0j)
0j
0j
0j
-0j
0j
0j
(-0+0j)
0j
0j
0j
0j
0j
0j
(-0+0j)
(-0+0j)


In [ ]:
test_dir = os.path.join(eeg_dir, 'test')

for label_type in ['left', 'right', 'down', 'back', 'up','forward']:
	dir_name = os.path.join(test_dir, label_type)
	for fname in os.listdir(dir_name):
		with open(os.path.join(dir_name, fname)) as f:
			df = pd.read_csv(f)
			for column in df:
				band_pass_eeg = butter_bandpass_filter(data = df[column], lowcut = 1,highcut = 50, fs = 250, order = 3)
				b_notch, a_notch = iirnotch(w0=59.9250936329588, Q=20, fs=250)
				filtered_eeg = filtfilt(b_notch, a_notch, band_pass_eeg)
				fft_data = fft(filtered_eeg)
				feature =  feature_extraction(fft_data)
				row = np.concatenate((row,feature),axis=None)
		result_test= result_test.append(pd.DataFrame(row).T)
		labels_test.append(label_type)
		row=[]

print(result_test)
			

(-0+0j)
0j
0j
(-0+0j)
0j
(-0+0j)
0j
(-0+0j)
(-0+0j)
-0j
(-0+0j)
(-0+0j)
(10+0j)
0j
-0j
(-0+0j)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:27: DeprecationWarning: The Python built-in `round` is deprecated for complex scalars, and will raise a `TypeError` in a future release. Use `np.round` or `scalar.round` instead.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:28: DeprecationWarning: The Python built-in `round` is deprecated for complex scalars, and will raise a `TypeError` in a future release. Use `np.round` or `scalar.round` instead.


(-0+0j)
(-0+0j)
0j
(-0+0j)
-0j
0j
0j
(-10+0j)
-0j
(-10+0j)
0j
(-0+0j)
0j
(-0+0j)
0j
-0j
-0j
0j
-0j
0j
(-0+0j)
-0j
-0j
0j
0j
-0j
0j
0j
0j
(-0+0j)
-0j
(-0+0j)
0j
0j
0j
0j
(-0+0j)
(-0+0j)
0j
0j
0j
0j
-0j
-0j
(-0+0j)
(-0+0j)
(-0+0j)
-0j
0j
-0j
-0j
0j
(-10-0j)
(-0+0j)
0j
(-0+0j)
0j
-0j
0j
0j
-0j
(-30+0j)
0j
0j
0j
(10+0j)
0j
0j
0j
0j
0j
-0j
(-0+0j)
-0j
0j
0j
(-0+0j)
0j
(-0+0j)
-0j
0j
0j
(-0+0j)
-0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
(-0+0j)
(-0+0j)
0j
(10+0j)
-0j
0j
0j
(-0+0j)
(-0+0j)
(-0+0j)
(-0+0j)
(-0+0j)
0j
0j
0j
(-0+0j)
-0j
0j
0j
0j
(-0+0j)
0j
0j
(-0+0j)
(-0+0j)
0j
(-0+0j)
(-0+0j)
(-0+0j)
(-10+0j)
(-0+0j)
0j
0j
(-0+0j)
-0j
0j
0j
0j
0j
-0j
-0j
-0j
(-0+0j)
0j
0j
(10+0j)
(50+0j)
0j
(-0+0j)
0j
0j
-0j
(-0+0j)
0j
0j
0j
0j
-0j
0j
0j
(-0+0j)
0j
0j
0j
0j
0j
-0j
0j
0j
0j
(-0+0j)
0j
0j
0j
(-0+0j)
0j
0j
0j
(-0+0j)
0j
0j
(-0+0j)
0j
(-0+0j)
0j
0j
0j
(-0+0j)
0j
-0j
(-0+0j)
(-0+0j)
0j
0j
(-10-0j)
(-0+0j)
(-0+0j)
0j
(-0+0j)
0j
0j
(-10+0j)
-0j
(-0+0j)
(-0+0j)
(-0+0j)
0j
(-0+0j)
(-0+0j)
0j
-0j
-0j
0j
-0j
(-

In [ ]:
result_train.shape

(120, 16)

In [ ]:
result_test.shape

(60, 16)

In [ ]:
len(labels_train)
			

120

In [ ]:
len(labels_test)

60

In [ ]:
from sklearn import preprocessing
# label_encoder object knows how to understand word labels.
label_encoder = preprocessing.LabelEncoder()
  

labels_train= label_encoder.fit_transform(labels_train)
labels_test = label_encoder.fit_transform(labels_test)

In [ ]:
print(labels_train)

[3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2]


In [ ]:
from sklearn.linear_model import Perceptron

ppn = Perceptron(eta0=0.1, random_state=1)
ppn.fit(result_train, labels_train)

ValueError: ignored